In [ ]:
!mkdir ./dogs-vs-cats
!mkdir ./dogs-vs-cats-redux-kernels-edition
!unzip  -o ../input/dogs-vs-cats/test1.zip -d ./dogs-vs-cats >a.txt
!unzip  -o ../input/dogs-vs-cats/train.zip -d ./dogs-vs-cats >a.txt
!unzip  -o ../input/dogs-vs-cats-redux-kernels-edition/train.zip -d ./dogs-vs-cats-redux-kernels-edition>a.txt
!unzip  -o ../input/dogs-vs-cats-redux-kernels-edition/test.zip -d ./dogs-vs-cats-redux-kernels-edition >a.txt

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
import torchvision.transforms as transforms
import torch.optim as optim
import torchvision.models as models
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from transformers import AdamW
from tqdm import tqdm, trange
from PIL import Image
from sklearn.decomposition import PCA

In [ ]:
trainLabel=[]
trianImagePaths=[]
for dirname, _, filenames in os.walk('./dogs-vs-cats/train'):
    for filename in filenames:
        trianImagePaths.append(os.path.join(dirname, filename))
        if filename.split('.')[0]=='cat':
            trainLabel.append(0)
        else:
            trainLabel.append(1)

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test, y_train, y_test = train_test_split(trianImagePaths,trainLabel,test_size=0.3, random_state=0)

In [ ]:
class DogsvsCats(Dataset):
    def __init__(self, imgDirList, labels=None, transform_apply=None, iftrain=True):
        self.imgDirList = imgDirList
        self.transform = transform_apply
        self.train = iftrain
        self.labels=labels
    def __len__(self):
        return len(self.imgDirList)
    
    def __getitem__(self, index):
        img = Image.open(self.imgDirList[index])
        
        if self.transform:
            img = self.transform(img)
        img=img[0,:,:].view(-1) 
        if self.train:
            label = torch.tensor(self.labels[index])
            return img,label
        else:
            return img

In [ ]:
train_transforms = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomHorizontalFlip(0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4884, 0.4551, 0.4170], std=[0.2256, 0.2210, 0.2214])
])

val_transforms = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4884, 0.4551, 0.4170], std=[0.2256, 0.2210, 0.2214])
])

In [ ]:
train_dataset = DogsvsCats(x_train,labels=y_train, transform_apply=train_transforms)
val_dataset = DogsvsCats(x_test,labels=y_test, transform_apply=val_transforms)

train_loader = torch.utils.data.DataLoader(train_dataset,batch_size=32,shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset,batch_size=32,shuffle=False)

In [ ]:
class myDNN(nn.Module):
    def __init__(self):
        super(myDNN,self).__init__()
        self.num_classes = 2
        self.fc1 = nn.Linear(224 * 224, 224*22)
        self.dropout1 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(224 * 22, 224*2)
    
        self.fc3 = nn.Linear(224 * 2, 224*2)
        self.dropout2 = nn.Dropout(0.1)
        self.classifier=nn.Linear(224*2, self.num_classes)

    def forward(self, x):
        x = self.dropout1(nn.functional.relu(self.fc1(x)))
        x=self.fc2(x)
        x=self.fc3(x)+x
        x=nn.functional.relu(x)
        feature=self.dropout2 (x)
        logits = self.classifier(feature)
        return logits

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model= myDNN()
model.to(device)
# 损失函数
criteon = nn.CrossEntropyLoss()
# 优化器
optimizer = AdamW(model.parameters(),lr=3e-5,eps=1e-8)

In [ ]:
def doEval(model,valid_dataloader):
    model.eval()
    eval_loss=0
    predictions, true_labels = [], []
    for batch in valid_dataloader:
        batch = tuple(t.to(device) for t in batch)
        data, label = batch
        with torch.no_grad():
            outputs = model(data)
            
        predictions.extend(torch.argmax(outputs, dim=-1))
        
        label_ids = label.to('cpu').numpy()
        true_labels.extend(label_ids)
        
        eval_loss += criteon(outputs, label).mean().item()
    eval_loss = eval_loss / len(valid_dataloader)
    rue_labels,predictions=torch.tensor(true_labels),torch.tensor(predictions)
    acc = accuracy_score(true_labels,predictions)
    f1 = f1_score(true_labels,predictions)
    recall = recall_score(true_labels,predictions)
    precision=precision_score(true_labels,predictions)
    return acc,precision,recall,f1,eval_loss

In [ ]:
def doTrain(model,train_loader):
    model.train()
    total_loss = 0
    predictions, true_labels = [], []
    for batch in train_loader:
        optimizer.zero_grad()
        batch = tuple(t.to(device) for t in batch)
        data, label = batch
        outputs = model(data)
        loss=criteon(outputs, label).mean()
        loss.backward()
        total_loss += loss.item()
        optimizer.step()
        predictions.extend(torch.argmax(outputs, dim=-1))
        label_ids = label.to('cpu').numpy()
        true_labels.extend(label_ids)
    train_loss = total_loss / len(train_loader)
    rue_labels,predictions=torch.tensor(true_labels),torch.tensor(predictions)
    acc = accuracy_score(true_labels,predictions)
    f1 = f1_score(true_labels,predictions)
    recall = recall_score(true_labels,predictions)
    precision=precision_score(true_labels,predictions)
    return acc,precision,recall,f1,train_loss

In [ ]:
acc_eva = []
f1_eva = []
recall_eva = []
precision_eva=[]

acc_train = []
f1_train = []
recall_train = []
precision_train=[]

loss_train=[]
loss_eva=[]
## Store the average loss after each epoch so we can plot them.
for _ in trange(20, desc="Epoch"):
    acc,precision,recall,f1,train_loss=doTrain(model,train_loader)
    acc_train.append(acc)
    f1_train.append(f1)
    recall_train.append(recall)
    precision_train.append(precision)
    loss_train.append(train_loss)
    print("Train Accuracy-Score: {}".format(acc))
    print("Train F1-Score: {}".format(f1))
    print("Train recall-Score: {}".format(recall))
    print("Train precision-Score: {}".format(precision))
    print("Average Train loss: {}".format(train_loss))
    print()
    
    acc,precision,recall,f1,eval_loss=doEval(model,val_loader)
    acc_eva.append(acc)
    f1_eva.append(f1)
    recall_eva.append(recall)
    precision_eva.append(precision)
    loss_eva.append(eval_loss)
    print("Validation Accuracy-Score: {}".format(acc))
    print("Validation F1-Score: {}".format(f1))
    print("Validation recall-Score: {}".format(recall))
    print("Validation precision-Score: {}".format(precision))
    print("Average Validation loss: {}".format(eval_loss))
    print()

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns

# Use plot styling from seaborn.
sns.set(style='darkgrid')

# Increase the plot size and font size.
sns.set(font_scale=1.5)
plt.rcParams["figure.figsize"] = (12,6)

# Plot the learning curve.
plt.plot(loss_train, '-o', label="training loss")
plt.plot(loss_eva, '-o', label="validation loss")
# Label the plot.
plt.title("Learning curve")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set(style='darkgrid')
sns.set(font_scale=1.5)
plt.rcParams["figure.figsize"] = (12,6)
plt.plot(acc_eva, 'b-o', label="evalation")
plt.plot(acc_train, 'r-o', label="target train")
plt.title("Acc curve")
plt.xlabel("Epoch")
plt.ylabel("acc")
plt.legend()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set(style='darkgrid')
sns.set(font_scale=1.5)
plt.rcParams["figure.figsize"] = (12,6)
plt.plot(f1_eva, 'b-o', label="evalation")
plt.plot(f1_train, 'r-o', label="target train")
plt.title("F1 curve")
plt.xlabel("Epoch")
plt.ylabel("f1")
plt.legend()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set(style='darkgrid')
sns.set(font_scale=1.5)
plt.rcParams["figure.figsize"] = (12,6)
plt.plot(recall_eva, 'b-o', label="evalation")
plt.plot(recall_train, 'r-o', label="target train")
plt.title("recall curve")
plt.xlabel("Epoch")
plt.ylabel("recall")
plt.legend()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set(style='darkgrid')
sns.set(font_scale=1.5)
plt.rcParams["figure.figsize"] = (12,6)
plt.plot(precision_eva, 'b-o', label="evalation")
plt.plot(precision_train, 'r-o', label="target train")
plt.title("precision curve")
plt.xlabel("Epoch")
plt.ylabel("precision")
plt.legend()
plt.show()